# <center> Домашнее задание №10
## <center> Градиентный бустинг

Ваша задача — побить как минимум 2 бенчмарка в этом [соревновании на Kaggle](https://www.kaggle.com/c/flight-delays-spring-2018). Здесь не будет подробных инструкций. Мы лишь кратко опишем, как был достигнут второй бенчмарк с помощью Xgboost. Надеемся, что на этом этапе курса достаточно бегло взглянуть на данные, чтобы понять: это тот тип задачи, где градиентный бустинг покажет себя хорошо. Скорее всего, это будет Xgboost, однако здесь много категориальных признаков.

<img src=https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg width=40% />

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv('../data/flight_delays_train.csv')
test = pd.read_csv('../data/flight_delays_test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

По времени вылета, коду перевозчика, аэропорту отправления, пункту назначения и дистанции перелёта необходимо предсказать задержку вылета более чем на 15 минут. В качестве простейшего бенчмарка возьмём классификатор Xgboost с двумя признаками, которые проще всего использовать: DepTime и Distance. Такая модель даёт 0.68202 на лидерборде.

In [ ]:
X_train = train[['Distance', 'DepTime']].values
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = \
    train_test_split(X_train, y_train, 
                     test_size=0.3, random_state=17)

Обучим Xgboost с параметрами по умолчанию на части данных и оценим ROC AUC на отложенной выборке.

In [ ]:
xgb_model = XGBClassifier(seed=17)

xgb_model.fit(X_train_part, y_train_part)
xgb_valid_pred = xgb_model.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, xgb_valid_pred)

Теперь сделаем то же самое на всей обучающей выборке, сделаем предсказания для тестовой выборки и сформируем файл для отправки. Так вы побьёте первый бенчмарк.

In [ ]:
xgb_model.fit(X_train, y_train)
xgb_test_pred = xgb_model.predict_proba(X_test)[:, 1]

pd.Series(xgb_test_pred, 
          name='dep_delayed_15min').to_csv('xgb_2feat.csv', 
                                           index_label='id', header=True)

Второй бенчмарк в лидерборде был достигнут следующим образом:

- Признаки `Distance` и `DepTime` были взяты без изменений
- Из признаков `Origin` и `Dest` был создан признак `Flight`
- Признаки `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и `Flight` были преобразованы с помощью OHE (`LabelBinarizer`)
- Были обучены логистическая регрессия и градиентный бустинг (xgboost). Гиперпараметры xgboost настраивались через кросс-валидацию. Сначала оптимизировались гиперпараметры, отвечающие за сложность модели, затем число деревьев было зафиксировано на 500, и настраивался шаг обучения.
- Предсказанные вероятности были получены через кросс-валидацию с помощью `cross_val_predict`. Линейная смесь предсказаний логистической регрессии и градиентного бустинга была задана в форме $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ — вероятность класса 1, предсказанная логистической регрессией, а $p_{xgb}$ — то же для xgboost. Вес $w_1$ подбирался вручную.
- Аналогичная комбинация предсказаний была сделана для тестовой выборки.

Следовать в точности этим шагам необязательно. Это лишь описание того, как результат был получен автором задания. Возможно, вы захотите пойти другим путём — например, добавить пару хороших признаков и обучить случайный лес из тысячи деревьев.

Удачи!